In [66]:
# Import Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model

# Load and preprocess the dataset
file_path = '/content/drive/My Drive/Colab Notebooks/pima-indians-diabetes.data.csv'
data = pd.read_csv(file_path, header=None)
data.columns = [
    'Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
    'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'
]

# Replace zeros with NaN for specific columns and fill with median
columns_with_zeros = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
data[columns_with_zeros] = data[columns_with_zeros].replace(0, np.nan)
data.fillna(data.median(), inplace=True)

# Standardize data
scaler = StandardScaler()
X = scaler.fit_transform(data.iloc[:, :-1])
y = data['Outcome']

# Handle class imbalance using SMOTE
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Build the improved neural network model
model = Sequential([
    Dense(128, input_dim=X_train.shape[1], activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping and learning rate reduction
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)
]

# Train the model
history = model.fit(X_train, y_train, epochs=150, batch_size=16, validation_split=0.2, callbacks=callbacks, verbose=1)

# Save the trained model
model.save("improved_diabetes_predictor_model.h5")

# Evaluate the model
y_pred = (model.predict(X_test) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Epoch 1/150


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, pref

35/35 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.5135 - loss: 0.8805 - val_accuracy: 0.7143 - val_loss: 0.6388 - learning_rate: 0.0010
Epoch 2/150
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6667 - loss: 0.6625 - val_accuracy: 0.7357 - val_loss: 0.5987 - learning_rate: 0.0010
Epoch 3/150
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7143 - loss: 0.6023 - val_accuracy: 0.7214 - val_loss: 0.5844 - learning_rate: 0.0010
Epoch 4/150
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7486 - loss: 0.5431 - val_accuracy: 0.7357 - val_loss: 0.5586 - learning_rate: 0.0010
Epoch 5/150
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7468 - loss: 0.4934 - val_accuracy: 0.7214 - val_loss: 0.5545 - learning_rate: 0.0010
Epoch 6/150
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7032 - loss: 0.6096 - val_accuracy: 0.7286 - val_loss: 0.5344 - learning_rate: 0.0010
Epoch 7/150
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7310 - loss: 0.5213 - val_accuracy: 0.73

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Accuracy: 0.77
Precision: 0.75
Recall: 0.83
F1 Score: 0.79


In [68]:
# Define the enhanced chatbot class
class MedAIChatbot:
    def __init__(self, model_path, scaler):
        self.model = load_model(model_path)
        self.scaler = scaler
        self.features = [
            "Pregnancies", "Glucose", "BloodPressure", "SkinThickness", "Insulin",
            "BMI", "DiabetesPedigreeFunction", "Age"
        ]
        # Predefined responses for common questions
        self.common_questions = {
            "services": "MedAI offers early disease detection, a diabetes symptom checker, personalized treatment plans, and expert healthcare consultations.",
            "diabetes symptom checker": "The diabetes symptom checker uses AI to analyze patient data for early detection of diabetes.",
            "appointment": "You can schedule an appointment by providing your preferred date and time. Type 'schedule' followed by your details.",
            "contact": "You can contact MedAI at contact@medai.com or call us at +1-800-555-MEDAI.",
            "medical advice": (
                "Here are some general tips:\n"
                "1. Maintain a balanced diet rich in fruits, vegetables, and lean proteins.\n"
                "2. Exercise regularly—at least 30 minutes of moderate activity per day.\n"
                "3. Stay hydrated and drink plenty of water.\n"
                "4. Get enough sleep (7-9 hours daily).\n"
                "5. Consult a doctor if you experience unusual symptoms or health issues."
            )
        }

    def handle_common_question(self, question):
        """Handle questions based on keywords."""
        keyword_responses = {
            ("service", "services", "medai"): self.common_questions["services"],
            ("symptom", "checker", "diabetes"): self.common_questions["diabetes symptom checker"],
            ("schedule", "appointment", "book appointment"): self.common_questions["appointment"],
            ("contact", "number", "phone", "contact details"): self.common_questions["contact"],
            ("tip", "health tip", "advice", "medical advice"): self.common_questions["medical advice"],
            ("bye", "exit"): "Goodbye!"
        }

        # Match user input to keywords
        for keywords, response in keyword_responses.items():
            if any(keyword in question for keyword in keywords):
                return response

        # Fallback for unmatched questions
        return "I'm sorry, I didn't understand your question. Can you please rephrase?"

    def handle_appointment(self, details):
        """Handle appointment scheduling."""
        return f"Appointment scheduled successfully with the following details: {details}"

    def get_response(self, user_input):
        if isinstance(user_input, dict):
            # Predictive response for diabetes check
            try:
                # Check for missing features
                missing_features = [feature for feature in self.features if feature not in user_input or user_input[feature] is None]
                if missing_features:
                    return f"Missing features: {', '.join(missing_features)}. Please provide all required inputs."

                # Extract and validate feature values
                values = [float(user_input[feature]) for feature in self.features]

                # Create a DataFrame with feature names
                input_data = pd.DataFrame([values], columns=self.features)
                values = []
                for feature in self.features:
                    value = user_input.get(feature)
                    values.append(float(value))

                input_data_scaled = self.scaler.transform([values])

                # Predict using the model
                prediction = self.model.predict(input_data_scaled)[0][0]
                diagnosis = "Diabetic" if prediction > 0.5 else "Non-Diabetic"
                return f"Prediction: {diagnosis}. Probability: {prediction:.2f}"

            except Exception as e:
                # Log exception (optional) and return a user-friendly message
                print(f"Error: {str(e)}")  # Log the error for debugging
                return "An error occurred while processing your data. Please try again."

        elif isinstance(user_input, str):
            # Lowercase for consistency
            question = user_input.lower().strip()
            if question.startswith("schedule"):
                # Extract appointment details by removing the keyword 'schedule'
                details = question[len("schedule"):].strip()  # Extract the rest of the string after 'schedule'
                # Check if the details are provided or empty
                if not details:
                    return "Please provide appointment details, such as the date and time (e.g., 'schedule 15th March at 3 PM')."
                # Optionally validate the details for basic formatting
                return self.handle_appointment(details)

            # Handle dynamic keyword-based questions
            return self.handle_common_question(question)

        else:
            return "Invalid input. Please provide a valid question or diabetes data."


# Instantiate the chatbot
#scaler = joblib.load("scaler.pkl")  # Ensure your scaler is loaded here (e.g., from joblib or pickle)
chatbot = MedAIChatbot("improved_diabetes_predictor_model.h5", scaler)

# Interact with the chatbot
print("MedAI Chatbot is ready! Type 'exit' or 'bye' to quit.")
while True:
    print("\nAsk a healthcare question or provide diabetes data as a JSON-like dictionary.")
    print("Example: {'Glucose': 120, 'BMI': 30.5, ...} or 'What are MedAI's services?'")
    user_input = input("User: ")
    if user_input.lower() in ["exit", "bye"]:
        print("Goodbye!")
        break
    try:
        # Determine if input is JSON-like or a string
        if user_input.startswith("{") and user_input.endswith("}"):
            user_data = eval(user_input)  # Convert string input to a Python dictionary
        else:
            user_data = user_input  # Treat as a general question
        response = chatbot.get_response(user_data)
        print("Bot:", response)
    except Exception as e:
        print("Bot: Invalid input. Please provide data in the correct format.")


MedAI Chatbot is ready! Type 'exit' or 'bye' to quit.

Ask a healthcare question or provide diabetes data as a JSON-like dictionary.
Example: {'Glucose': 120, 'BMI': 30.5, ...} or 'What are MedAI's services?'
User: bye
Goodbye!
